This notebook demos `PromptTypeWrapper`, a transformer that produces abstract representations of an utterance in terms of its phrasing and its rhetorical intent. 

The transformer, with some minor modifications, implements the methodology detailed in the [paper](http://www.cs.cornell.edu/~cristian/Asking_too_much.html), 

```
Asking Too Much? The Rhetorical Role of Questions in Political Discourse 
Justine Zhang, Arthur Spirling, Cristian Danescu-Niculescu-Mizil
Proceedings of EMNLP 2017
```

and by default analyzes _questions_ and their responses (though this can be modified on initialization). 

Under the surface, the transformer implements two key modules, `PhrasingMotifs` and `PromptTypes`, as well as a suite of preprocessing steps. For a more detailed description of each of these steps, and examples of calling the component modules separately, see [this notebook](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/prompt-types/prompt-type-demo.ipynb).

First we load the corpus. We will examine a dataset of questions from question periods that take place in the British House of Commons (also detailed in the paper). 

In [24]:
from convokit import Corpus
from convokit import download
from convokit.prompt_types import PromptTypeWrapper

For expedience, we load pre-computed dependency parses, which should come with the data release (see [this notebook](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/text-processing/text_preprocessing_demo.ipynb) for a demonstration of how to get these parses for yourself).

In [3]:
# OPTION 1: DOWNLOAD CORPUS 
# UNCOMMENT THESE LINES TO DOWNLOAD CORPUS
# DATA_DIR = '<YOUR DIRECTORY>'
# ROOT_DIR = download('parliament-corpus', data_dir=DATA_DIR)

# OPTION 2: READ PREVIOUSLY-DOWNLOADED CORPUS FROM DISK
# UNCOMMENT THIS LINE AND REPLACE WITH THE DIRECTORY WHERE THE PARLIAMENT-CORPUS IS LOCATED
# ROOT_DIR = '<YOUR DIRECTORY>'

corpus = Corpus(ROOT_DIR)
corpus.load_info('utterance',['parsed'])

In [4]:
VERBOSITY = 10000

Inspecting an example utterance:

In [5]:
test_utt_id = '1997-01-27a.4.0'
utt = corpus.get_utterance(test_utt_id)

In [6]:
utt.text

"Does my right hon Friend agree that last week 's statement about a replacement royal yacht has been widely welcomed ? Does he agree also that , ideally , Britannia should become the centrepiece of the millennium project in Portsmouth harbour , spanning Gosport and Portsmouth ? I am sure that that idea would prove very popular . As to plans for a new yacht , does my right hon Friend share my distaste for the Opposition 's tactics ? They had every opportunity to express their grudging and negative attitude during the past two years when the project was under discussion ."

Initializing a `PromptTypeWrapper` model, that will infer 8 types of questions (see docstring for other arguments):

In [7]:
pt = PromptTypeWrapper(n_types=8, random_state=1000)

In [8]:
pt.fit(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

	counting itemset cooccurrences for 90000/318345 collections
	counting itemset cooccurrences for 100000/318345 collections
	counting itemset cooccurrences for 110000/318345 collections
	counting itemset cooccurrences for 120000/318345 collections
	counting itemset cooccurrences for 130000/318345 collections
	counting itemset cooccurrences for 140000/318345 collections
	counting itemset cooccurrences for 150000/318345 collections
	counting itemset cooccurrences for 160000/318345 collections
	counting itemset cooccurrences for 170000/318345 collections
	counting itemset cooccurrences for 180000/318345 collections
	counting itemset cooccurrences for 190000/318345 collections
	counting itemset cooccurrences for 200000/318345 collections
	counting itemset cooccurrences for 210000/318345 collections
	counting itemset cooccurrences for 220000/318345 collections
	counting itemset cooccurrences for 230000/318345 collections
	counting itemset cooccurrences for 240000/318345 collections
	counting

Output. Note that this should produce the same output as calling the component transformers separately, as detailed in [this notebook](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/prompt-types/prompt-type-wrapper-demo.ipynb):

In [9]:
for i in range(8):
    print(i)
    pt.display_type(i,  k=15)
    print('\n\n')

0
top prompt:


,0,1,2,3,4,5,6,7,type_id
made_*,0.627821,1.112966,1.253935,1.080853,1.263088,1.081064,1.085615,1.120296,0.0
made_*__made_in,0.670131,1.092172,1.296672,1.044795,1.165740,1.096866,1.117699,1.102888,0.0
made_*__made_to,0.677337,1.226368,1.219402,1.145383,1.388485,1.110612,1.212821,1.180075,0.0
in>*__tell_*,0.681406,1.176855,0.986156,1.121633,1.332023,0.847615,0.987093,1.266240,0.0
made_*__made_what,0.683845,1.124455,1.353602,1.139765,1.248646,1.112601,1.209657,0.959844,0.0
made_*__made_been,0.689245,1.122980,1.277784,1.117208,1.263149,1.144714,1.177270,1.178932,0.0
happen_*__happen_will,0.697615,1.202465,1.101319,1.157835,1.233026,0.868512,1.052773,1.120683,0.0
made_*__what>*,0.698273,1.148946,1.360852,1.140067,1.247890,1.156193,1.226739,0.997630,0.0
made_*__made_been__made_what,0.706422,1.105542,1.336376,1.133772,1.224077,1.158618,1.230570,1.052753,0.0
made_*__made_has,0.707376,1.123585,1.303389,1.181350,1.307120,1.138856,1.224905,1.109336,0.0


top response:


,0,1,2,3,4,5,6,7,type_id
am_at,0.744773,0.935439,1.205010,1.062946,1.220152,1.025650,1.073168,1.238502,0.0
known_*,0.746823,1.227288,1.121113,1.157730,1.273498,1.041451,1.067706,1.242575,0.0
can_*,0.785983,1.107881,1.200609,1.058894,1.083504,0.993568,1.070240,0.982096,0.0
place_*,0.789185,1.049869,1.162727,1.044718,1.156990,1.036115,1.094712,1.125281,0.0
assure_have,0.796057,0.944208,1.301825,0.915244,0.966593,1.071265,1.028048,0.994812,0.0
was_made,0.797752,1.188686,0.977958,1.147366,1.179836,0.915078,0.944907,1.195879,0.0
make_shall,0.802670,0.873247,1.176298,0.898143,0.952539,0.898050,0.834410,1.129150,0.0
give_can,0.804878,1.036402,1.154003,1.068707,1.223465,0.858061,1.072920,1.131898,0.0
have_made,0.811640,1.045384,1.182388,0.922776,1.063783,1.034568,0.824843,1.211438,0.0
write_shall,0.815864,1.160921,1.133396,1.170030,1.271814,1.098569,1.200611,1.185541,0.0





1
top prompt:


...

,0,1,2,3,4,5,6,7,type_id
realise_*,1.051353,1.040419,0.870504,0.830392,0.898416,0.891873,0.510753,1.333201,6.0
therefore>*,1.073179,1.107272,0.791517,0.893782,0.895427,0.865649,0.533603,1.356722,6.0
realise_is,1.076871,1.040312,0.900864,0.878094,0.820099,0.953101,0.537008,1.288549,6.0
be_right,0.983154,0.999825,1.011520,0.824180,0.882781,0.827222,0.597474,1.218149,6.0
be_however,1.001768,0.831704,1.087172,0.702314,0.746173,0.884701,0.599450,1.203639,6.0
remind_is,1.095861,0.955227,0.928311,0.856989,0.948804,0.962759,0.601668,1.334839,6.0
be_might,1.103935,0.896718,1.006554,0.767808,0.674502,0.895251,0.602006,1.173772,6.0
believe_however,1.055552,0.951726,1.031753,0.781682,0.738250,0.912921,0.602504,1.207242,6.0
be_decide,1.006482,1.026823,0.967803,0.891903,0.886792,0.873450,0.603566,1.237801,6.0
realise_will,1.032995,1.115808,0.867645,0.958823,1.015834,0.951978,0.605154,1.362984,6.0





7
top prompt:


,0,1,2,3,4,5,6,7,type_id
doing_*__what>*,1.188908,1.179227,1.296108,1.284275,1.161041,1.174012,1.380595,0.487743,7.0
doing_*,1.196526,1.177769,1.272685,1.269288,1.144117,1.162529,1.344962,0.501558,7.0
taking_*__taking_is__what>*,1.126557,1.190315,1.332255,1.244363,1.160392,1.187807,1.405707,0.508016,7.0
doing_*__doing_is__what>*,1.194910,1.201274,1.301613,1.311563,1.213732,1.199367,1.419399,0.529425,7.0
take_*__take_what,1.156225,1.012067,1.335862,1.188823,1.090698,1.169019,1.304218,0.532164,7.0
taking_*__taking_are,1.087618,1.218423,1.353556,1.237473,1.191860,1.198082,1.402136,0.533785,7.0
taking_*,1.134108,1.230914,1.339015,1.254403,1.189327,1.215009,1.418624,0.534918,7.0
will>*__work_*__work_with,1.055845,0.969658,1.376481,1.118873,1.019038,1.158587,1.261221,0.535301,7.0
taking_*__what>*,1.084641,1.223158,1.355838,1.253558,1.196290,1.203940,1.413722,0.540402,7.0
doing_*__doing_is,1.204739,1.219920,1.278352,1.299628,1.208292,1.188003,1.396126,0.541201,7.0


top response:


,0,1,2,3,4,5,6,7,type_id
through>*,1.173298,1.242311,1.328201,1.279822,1.054140,1.258116,1.369618,0.642623,7.0
is_working,1.119604,1.152739,1.275667,1.203138,0.976927,1.155934,1.216139,0.645541,7.0
ensuring_is,1.195901,1.089027,1.243815,1.110770,0.954979,1.149191,1.229360,0.648673,7.0
supporting_are,1.214342,1.245588,1.238312,1.291103,1.178745,1.182181,1.357689,0.653380,7.0
working_on,1.134486,1.224916,1.291671,1.280997,1.308231,1.218774,1.380054,0.664101,7.0
supporting_*,1.220306,1.258811,1.250189,1.318995,1.196743,1.203403,1.381304,0.666979,7.0
ensuring_*,1.175195,1.123449,1.225120,1.152106,0.926351,1.104832,1.186094,0.669527,7.0
working_are,1.136606,1.229650,1.284925,1.282427,1.311925,1.214676,1.377935,0.670170,7.0
working_with,1.136615,1.228151,1.286453,1.281580,1.314418,1.215308,1.379528,0.672401,7.0
working_*,1.138597,1.230087,1.284754,1.282966,1.312211,1.216490,1.379105,0.672730,7.0


Transforming a single utterance. The model will annotate each utterance with a set of rerpesntations or features.

In [10]:
utt = pt.transform_utterance(utt)

the phrasing motifs, i.e., a representation of how each sentence in the utterance is phrased:

In [11]:
utt.get_info('motifs')

['agree_* agree_*__does>* does>*',
 'agree_* agree_*__agree_also agree_*__does>* does>*',
 'as>* share_* share_*__share_does']

A vector representation encapsulating the utterance's rhetorical intent (in short, an embedding of the utterance based on the responses associated with questions containing its constituent phrasings. see paper for details):

In [12]:
utt.get_info('prompt_types__prompt_repr')

[-0.17102469270226672,
 0.030668340328548496,
 -0.14361175504485915,
 0.11035667766433878,
 -0.3149225651493796,
 -0.032341849264302266,
 -0.22282059450017552,
 -0.12806097960998153,
 0.1771197712150501,
 0.02081981426950371,
 -0.3536289362524308,
 -0.2403882732149338,
 -0.06125687797720545,
 -0.19491483622907677,
 -0.05056529746639132,
 -0.03309523304250256,
 -0.41507809747180324,
 -0.06014879946683283,
 -0.11378928189631009,
 -0.01750400434622636,
 -0.04641636672373653,
 -0.5430246645922729,
 0.13134581675643944,
 -0.0851526519739321]

Distances between that vector and the centroid of each inferred cluster

In [13]:
utt.get_info('prompt_types__prompt_dists.8')

[1.1430428918990696,
 0.9502324189972602,
 1.1337070117657726,
 0.8403522127994898,
 0.38907971018922277,
 1.1182399779615004,
 0.7633996580496394,
 1.1170915977638758]

The particular type of question, and how close it is to the centroid of that particular cluster:

In [14]:
utt.get_info('prompt_types__prompt_type.8')

4.0

In [15]:
utt.get_info('prompt_types__prompt_type_dist.8')

0.38907971018922277

Transforming the entire corpus:

In [16]:
corpus = pt.transform(corpus)

10000/433787 utterances processed
20000/433787 utterances processed
30000/433787 utterances processed
40000/433787 utterances processed
50000/433787 utterances processed
60000/433787 utterances processed
70000/433787 utterances processed
80000/433787 utterances processed
90000/433787 utterances processed
100000/433787 utterances processed
110000/433787 utterances processed
120000/433787 utterances processed
130000/433787 utterances processed
140000/433787 utterances processed
150000/433787 utterances processed
160000/433787 utterances processed
170000/433787 utterances processed
180000/433787 utterances processed
190000/433787 utterances processed
200000/433787 utterances processed
210000/433787 utterances processed
220000/433787 utterances processed
230000/433787 utterances processed
240000/433787 utterances processed
250000/433787 utterances processed
260000/433787 utterances processed
270000/433787 utterances processed
280000/433787 utterances processed
290000/433787 utterances proc

Other examples:

In [17]:
utt1 = corpus.get_utterance('1987-03-04a.857.5')

In [18]:
utt1.get_info('motifs')

['stop_* stop_*__stop_will stop_*__stop_will__will>* stop_*__will>* will>*',
 'admit_* admit_*__admit_will admit_*__admit_will__will>* admit_*__will>* will>*',
 'does>* does>*__does>not does>*__understand_* understand_* understand_*__understand_does']

In [19]:
utt1.text

'Will the Secretary of State stop giving us what is called in the pop record industry a remix of alibis , excuses and gimmicks ? Will he admit that the number of homes built to rent last year by local authorities was the lowest in 62 years , that the housing investment programme net of capital receipts was the lowest in real terms since HIPs were invented and that , even during the past three years the number of repair and improvement grants , which would bring some private homes back into use , have dropped by 100,000 ? Does not the right hon Gentleman understand that , if the private owner and the local authority are starved of resources , we are left with lengthy queues , homelessness and all the other scandals of poor housing that exist today ?'

In [20]:
utt1.get_info('prompt_types__prompt_type.8')

2.0

We can also try out the model on arbitrary input. For instance, we see that the following question is also of type 5 -- that is, similar to other questions which voice agreement or support.

In [21]:
str_utt = pt.transform_utterance('Do you share my distaste for cockroaches?')

In [22]:
str_utt.get_info('motifs')

['do>* share_*']

In [23]:
str_utt.get_info('prompt_types__prompt_type.8')

4.0

Serializing the model. This dumps both the underlying `PhrasingMotifs` and `PromptTypes` models to disk:

In [21]:
import os

In [23]:
pt.dump_models(os.path.join(ROOT_DIR, 'full_pipe_models'))

writing itemset counts
writing downlinks
writing itemset to ids
writing meta information
dumping embedding model
dumping training embeddings
dumping type model 8


The entire pipeline can later be loaded back from memory and used to transform new data:

In [26]:
new_pt = PromptTypeWrapper(output_field='prompt_types_new',
                           min_support=100, svd__n_components=25, random_state=1000)

In [27]:
new_pt.load_models(os.path.join(ROOT_DIR, 'full_pipe_models'))

reading itemset counts
reading downlinks
reading itemset to ids
reading meta information
loading embedding model
loading training embeddings
loading type model 8


In [29]:
pt_model_dir = os.path.join(ROOT_DIR, 'full_pipe_models')
!ls $pt_model_dir

pm_model  pt_model


In [39]:
new_str_utt = new_pt.transform_utterance('Do you share my distaste for cockroaches?')

In [40]:
new_str_utt.get_info('motifs')

['do>* share_*']

In [41]:
new_str_utt.get_info('prompt_types__prompt_type.8')